In [124]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [125]:
dataset = pd.read_csv('../../data/funda_train.csv')

In [126]:
dataset['year_month'] = dataset['transacted_date'].str[0:7]

In [127]:
drop_features = ['installment_term', 'region', 'type_of_business','transacted_date','card_company','transacted_time']
dataset = dataset.drop(drop_features, axis=1)

In [128]:
subset = dataset[['store_id','amount','year_month','card_id']]
subset

refund = subset[subset['amount'] < 0]
refund

,store_id,amount,year_month,card_id
41,0,-8571.428571,2016-06,40
347,0,-1857.142857,2016-08,285
731,0,-2000.000000,2016-10,473
831,0,-85.714286,2016-11,230
944,0,-57.142857,2016-11,138
...,...,...,...,...
6556242,2136,-13428.571429,2019-02,4663626
6556448,2136,-6928.571429,2019-02,4663760
6556485,2136,-5571.428571,2019-02,4663779
6556489,2136,-8571.428571,2019-02,4663780


In [129]:
dataset.iloc[refund.index[0] -1 : refund.index[0] + 1]

,store_id,card_id,amount,year_month
40,0,40,8571.428571,2016-06
41,0,40,-8571.428571,2016-06


In [130]:
refund_case_1 = []
for i in tqdm(range(len(refund.index))):
    subset = dataset.iloc[refund.index[i]-1 : refund.index[i] + 1]
    if subset['amount'].sum() == 0:
        refund_case_1.append(refund.index[i]) # 환불 건의 index 를 추출합니다. index 41과 같은 건을 모두 추출
    else:
        pass

100%|██████████| 73100/73100 [00:12<00:00, 5763.40it/s]


In [131]:
refund_case_2 = []
for i in tqdm(range(len(refund.index))):
    subset = dataset.iloc[refund.index[i]-1 : refund.index[i] + 1]
    if subset['amount'].sum() == 0:
        refund_case_2.append(refund.index[i]-1)  # 환불 전 결제 건의 index 를 추출합니다. index 40과 같은 건을 모두 추출
    else:
        pass

100%|██████████| 73100/73100 [00:13<00:00, 5568.45it/s]


In [132]:
print("refund_case_1 : {refund} refund_case_2 : {payment}".format(refund = len(refund_case_1), payment = len(refund_case_2)))
if len(refund_case_1) == len(refund_case_2):
    print("두 케이스 동일")

refund_case_1 : 55347 refund_case_2 : 55347
두 케이스 동일


In [133]:
sub_refund = refund.drop(index = refund_case_1, axis = 0)

print("기존 환불 건수 : {before} 드랍 후 환불 건수 : {after}".format(before = len(refund), after = len(sub_refund)))

기존 환불 건수 : 73100 드랍 후 환불 건수 : 17753


In [134]:
sub_dataset = dataset.drop(index = refund_case_2, axis = 0)

In [138]:
refund_case_3 = []
for i in tqdm(range(len(sub_refund.index))):
    subset = sub_dataset.loc[(sub_dataset['amount'] == abs(sub_refund.iloc[i].amount)) & (sub_dataset['card_id'] == (sub_refund.iloc[i].card_id))] # 환불 전 결제 케이스
    refund_case_3.append(subset.index.min())

 12%|█▏        | 2189/17753 [01:01<07:17, 35.61it/s]


KeyboardInterrupt: 

In [ ]:
refund_case_3 = [x for x in refund_case_3 if np.isnan(x) == False]

- -- -------------

In [ ]:
refund_case_4 = list(sub_refund.index)

In [111]:
print("refund_case_3 : {before} refund_case_4 : {after}".format(before = len(refund_case_3), after = len(refund_case_4))) # 환불 케이스

refund_case_3 : 17695 refund_case_4 : 17753


In [117]:
len(dataset) - 73100*2 # 우리가 원하는 결과

6410413

In [119]:
dataset = dataset.drop(index = refund_case_1, axis = 0)
len(dataset)

6501266

In [120]:
dataset = dataset.drop(index = refund_case_2, axis = 0)
len(dataset)

6445919

In [121]:
dataset = dataset.drop(index = refund_case_3, axis = 0)
len(dataset)

KeyError: '[  37672   56265   70902   78330   86364  109911  126793  128444  128444\n  128664  128444  130980  131220  139162  165324  208142  208142  224290\n  225373  268626  278206  282475  282475  299113  328452  336058  365217\n  423900  440106  462401  469907  481943  524784  526965  604328  604328\n  604403  637966  648538  648538  648538  694350  711452  711452  798020\n  804604  805742  805950  806077  811127  811127  812232  837916  837916\n  837916  837916  837916  857482  858882  860630  873314  889862  911731\n  939301  949416 1020407 1020578 1029299 1035393 1035403 1054271 1061902\n 1077470 1077826 1091261 1114335 1116782 1152462 1152948 1152954 1152954\n 1153282 1154137 1173143 1180982 1188314 1190838 1190838 1191632 1248655\n 1248655 1359964 1361986 1377111 1378756 1378756 1378756 1378756 1378756\n 1378756  187642 1436445 1440836 1448166 1449288 1460050 1474187 1486299\n 1486991 1567971 1586542 1593405 1593646 1593706 1593722 1593801 1593855\n 1593961 1594071 1594848 16

In [122]:
dataset = dataset.drop(index = refund_case_4, axis = 0)
len(dataset)

6428166